In [1]:
from keras.datasets import mnist
from keras.models import Sequential, Model
from keras.layers import Dense, Dropout, Activation, Flatten, BatchNormalization, merge, Input
from keras.layers import Convolution2D, MaxPooling2D, ZeroPadding2D, AveragePooling2D, GlobalAveragePooling2D
from keras.utils import np_utils
from keras.models import model_from_json
from keras import backend as K
from keras.preprocessing import image
from keras.optimizers import SGD
from keras.utils.data_utils import get_file
import random
import os
import cv2

import numpy as np
import tensorflow as tf
import matplotlib.pyplot as plt
from keras.utils import to_categorical, plot_model 
%matplotlib inline

In [2]:
import glob 
import pandas as pd 
import re 
import time 
import itertools 
import seaborn as sns  
import keras 
import datetime

from numpy import expand_dims
from keras import callbacks 
from keras import layers
from keras.layers.normalization import BatchNormalization 
from keras import backend as k 
from keras import regularizers  
from keras.layers import Activation, add, Conv2D, Conv2DTranspose, Dense, Dropout, Flatten, Input, Lambda, MaxPooling2D, Reshape, ZeroPadding2D, AveragePooling2D 
from keras.layers.convolutional import UpSampling2D 
from keras.losses import binary_crossentropy 
from keras.models import Model, Sequential 
from keras.preprocessing.image import ImageDataGenerator 
from keras.utils import to_categorical, plot_model 
from keras.preprocessing.image import load_img 
from keras.preprocessing.image import img_to_array 
from sklearn.metrics import confusion_matrix, plot_confusion_matrix 
from timeit import default_timer as timer
from keras.models import Model
from keras.initializers import glorot_uniform

In [3]:
from keras.layers import Flatten, Conv2D, AveragePooling2D, MaxPooling2D, GlobalMaxPooling2D
from keras.models import Model, load_model
from keras.initializers import glorot_uniform

In [4]:
physical_devices = tf.config.experimental.list_physical_devices('GPU')
print("physical_devices-------------", len(physical_devices))
tf.config.experimental.set_memory_growth(physical_devices[0], True)

physical_devices------------- 1


In [5]:
ROWS = 224
COLS = 224
CHANNELS = 3
CLASSES = 30

In [6]:
def read_image(file_path): # Read images and convert it into the uniform size (224x224x3)
    img_start = cv2.imread(file_path, cv2.IMREAD_COLOR)
    return cv2.resize(img_start, (ROWS, COLS), interpolation=cv2.INTER_CUBIC)

In [7]:
filenames= os.listdir ('./images')
x_train = []
y_train = []
for folder in filenames: # Read the file name.
    for image in os.listdir(f'./images/{folder}'):# Use folder as parameter.
        img = read_image(f'./images/{folder}/{image}') # Read the files in each folder.
        x_train.append(img) # Append them (the file and label) in exact same order.
        y_train.append(folder)

In [8]:
c = list(zip(x_train, y_train)) # build the relationship between image and label
random.shuffle(c) # shuffle
a, b = zip(*c) # get image and label back (the order of rows is changed but the label is always attached to image)

In [9]:
from sklearn import preprocessing
label_encoder = preprocessing.LabelEncoder()

In [10]:
label_encoder.fit(["African elephant", "ant", "asparagus", "bridge", "computer keyboard","cow","dining table","Ferris wheel","flower","French bulldog","gas pump","gorilla","hula-hoop","jelly bean","jellyfish","king penguin","kiwi","koala","limousine","monarch","motorcycling","rodent","roller coaster","seashore","sea slug","sewing machine","skyscraper","snail","tiger","wasp's nest"])

LabelEncoder()

In [11]:
b_label = np.copy(b)
b_label = label_encoder.transform(b_label)

In [12]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(a, b_label, train_size=0.8,test_size=0.2, random_state=101)

In [13]:
training_image = np.copy(X_train)
training_label = np.copy(y_train)
test_image = np.copy(X_test)
test_label = np.copy(y_test)

training_image = np.array(training_image)
training_label = np.array(training_label)
test_image = np.array(test_image)
test_label = np.array(test_label)

In [14]:
training_images_norm = cv2.normalize(training_image, None, alpha=0, beta=1, norm_type=cv2.NORM_MINMAX, dtype=cv2.CV_32F)
testing_images_norm = cv2.normalize(test_image, None, alpha=0, beta=1, norm_type=cv2.NORM_MINMAX, dtype=cv2.CV_32F)

In [15]:
x_label_OneHot = to_categorical(training_label, 30)
y_label_OneHot = to_categorical(test_label, 30)

In [16]:
print(np.shape(training_images_norm))

(24000, 224, 224, 3)


In [17]:
def identity_block(X, f, filters, stage, block):
    F1, F2, F3 = filters
    X_replica = X

    conv_name = 'res' + str(stage) + block + '_branch'
    bnorm_name = 'bnorm' + str(stage) + block + '_branch'

    X = Conv2D(F1, (1,1), strides= (1,1), padding= 'valid', name = conv_name + '2a', kernel_initializer= glorot_uniform(seed=0))(X)
    X = BatchNormalization(axis= 3, name = bnorm_name + '2a')(X)
    X = Activation('relu')(X)

    X = Conv2D(F2, (f,f), strides= (1,1), padding= 'same', name = conv_name + '2b', kernel_initializer= glorot_uniform(seed=0))(X)
    X = BatchNormalization(axis= 3, name = bnorm_name + '2b')(X)
    X = Activation('relu')(X)

    X = Conv2D(F3, (1,1), strides= (1,1), padding= 'valid', name = conv_name + '2c', kernel_initializer= glorot_uniform(seed=0))(X)
    X = BatchNormalization(axis= 3, name = bnorm_name + '2c')(X)

    X = Add()([X, X_replica])
    X = Activation('relu')(X)

    return X

In [18]:
#convolutional_block
def convolutional_block(X, f, filters, stage, block, s = 2):
    F1, F2, F3 = filters
    X_replica = X

    conv_name = 'res' + str(stage) + block + '_branch'
    bnorm_name = 'bnorm' + str(stage) + block + '_branch'

    X = Conv2D(F1, (1,1), strides= (s,s), name = conv_name + '2a', kernel_initializer= glorot_uniform(seed = 0))(X)
    X = BatchNormalization(axis = 3, name= bnorm_name + '2a')(X)
    X = Activation('relu')(X)

    X = Conv2D(F2, (f,f), strides= (1,1), name = conv_name + '2b', padding= 'same', kernel_initializer= glorot_uniform(seed = 0))(X)
    X = BatchNormalization(axis = 3, name= bnorm_name + '2b')(X)
    X = Activation('relu')(X)

    X = Conv2D(F3, (1,1), strides= (1,1), name = conv_name + '2c', padding= 'valid', kernel_initializer= glorot_uniform(seed = 0))(X)
    X = BatchNormalization(axis = 3, name= bnorm_name + '2c')(X)

    X_replica = Conv2D(F3, (1,1), strides= (s,s), name = conv_name + '1', padding='valid', kernel_initializer= glorot_uniform(seed = 0))(X_replica)
    X_replica = BatchNormalization(axis = 3, name = bnorm_name + '1')(X_replica)

    X = Add()([X, X_replica])
    X = Activation('relu')(X)

    return X

In [19]:
# ResNet model
def ResNet50(input_shape= (224,224,3), classes=30):
    X_input = Input(input_shape)
    X = ZeroPadding2D(padding= (3,3))(X_input)

    #stage 1
    X = Conv2D(64, (7,7), strides= (2,2), name= 'conv1', kernel_initializer=glorot_uniform(seed=0))(X)
    X = BatchNormalization(axis = 3, name= 'bn_conv1')(X)
    X = Activation('relu')(X)
    X = MaxPooling2D((3,3), strides= (2,2))(X)

    #stage 2
    X = convolutional_block(X, f=3, filters= [64,64,256], stage=2, block= 'a', s=1)
    X = identity_block(X, f=3, filters= [64,64,256], stage=2, block='b')
    X = identity_block(X, f=3, filters= [64,64,256], stage=2, block='c')

    #stage 3
    X = convolutional_block(X, f=3, filters= [128,128,512], stage=3, block= 'a', s=2)
    X = identity_block(X, f=3, filters= [128,128,512], stage=3, block='b')
    X = identity_block(X, f=3, filters= [128,128,512], stage=3, block='c')
    X = identity_block(X, f=3, filters= [128,128,512], stage=3, block='d')

    #stage 4
    X = convolutional_block(X, f = 3, filters = [256, 256, 1024], stage=4, block='a', s = 2)
    X = identity_block(X, f= 3, filters= [256, 256, 1024], stage=4, block='b')
    X = identity_block(X, f= 3, filters= [256, 256, 1024], stage=4, block='c')
    X = identity_block(X, f= 3, filters= [256, 256, 1024], stage=4, block='d')
    X = identity_block(X, f= 3, filters= [256, 256, 1024], stage=4, block='e')
    X = identity_block(X, f= 3, filters= [256, 256, 1024], stage=4, block='f')

    #stage 5
    X = convolutional_block(X, f = 3, filters = [512, 512, 2048], stage=5, block='a', s = 2)
    X = identity_block(X, f= 3, filters= [512, 512, 2048], stage=5, block='b')
    X = identity_block(X, f= 3, filters= [512, 512, 2048], stage=5, block='c')

    X = AveragePooling2D()(X)

    X = Flatten()(X)
    X = Dense(classes, activation= 'softmax', name='fc' + str(classes), kernel_initializer= glorot_uniform(seed= 0))(X)

    model = Model(inputs = X_input, outputs = X, name='ResNet50')
    return model

In [20]:
model = ResNet50()

In [21]:
model.compile(loss='categorical_crossentropy', optimizer='sgd', metrics=['accuracy'])
model.summary()

Model: "ResNet50"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            [(None, 224, 224, 3) 0                                            
__________________________________________________________________________________________________
zero_padding2d (ZeroPadding2D)  (None, 230, 230, 3)  0           input_1[0][0]                    
__________________________________________________________________________________________________
conv1 (Conv2D)                  (None, 112, 112, 64) 9472        zero_padding2d[0][0]             
__________________________________________________________________________________________________
bn_conv1 (BatchNormalization)   (None, 112, 112, 64) 256         conv1[0][0]                      
___________________________________________________________________________________________

In [22]:
history = model.fit(training_images_norm, x_label_OneHot, batch_size=32, epochs=100,
                     validation_data=(testing_images_norm, y_label_OneHot), verbose=1)

Epoch 1/100
750/750 [==============================] - 142s 190ms/step - loss: 3.6444 - accuracy: 0.1892 - val_loss: 2.3998 - val_accuracy: 0.3113
Epoch 2/100
750/750 [==============================] - 139s 186ms/step - loss: 2.2465 - accuracy: 0.3512 - val_loss: 2.1279 - val_accuracy: 0.3875
Epoch 3/100
750/750 [==============================] - 141s 187ms/step - loss: 1.9559 - accuracy: 0.4340 - val_loss: 1.9493 - val_accuracy: 0.4418
Epoch 4/100
750/750 [==============================] - 143s 190ms/step - loss: 1.7526 - accuracy: 0.4888 - val_loss: 1.8544 - val_accuracy: 0.4637
Epoch 5/100
750/750 [==============================] - 143s 191ms/step - loss: 1.5644 - accuracy: 0.5372 - val_loss: 2.0409 - val_accuracy: 0.4323
Epoch 6/100
750/750 [==============================] - 142s 189ms/step - loss: 1.3867 - accuracy: 0.5877 - val_loss: 1.6536 - val_accuracy: 0.5320
Epoch 7/100
750/750 [==============================] - 144s 192ms/step - loss: 1.2322 - accuracy: 0.6306 - val_loss: 1